In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102
import os
import numpy as np

In [2]:
# Load the Flowers102 dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Download the dataset (training split)
train_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='train',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (validation split)
val_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='val',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

# Download the dataset (test split)
test_dataset = Flowers102(
    root='./data',  # The root directory where the dataset will be saved
    split='test',   # 'train' for the training set, 'test' for the test set
    transform=transform,  # Apply the defined transformation
    download=True  # Download if not already present
)

In [3]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [4]:
# Define the MixUp function
def mixup_data(x, y, alpha=1.0):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

In [5]:
# Create a MixUp data loader
def mixup_loader(dataloader, alpha=1.0):
    for data, target in dataloader:
        data, target_a, target_b, lam = mixup_data(data, target, alpha)
        yield data, target_a, target_b, lam

In [6]:
# Define the ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

# Modify the output layer to match the number of classes in the dataset (102 for Oxford Flowers)
model.fc = nn.Linear(model.fc.in_features, 102)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/codespace/.python/current/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Training loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

no_improvement_counter = 0
patience = 5
for epoch in range(num_epochs):
    model.train()
    total = 0
    correct = 0
    running_loss = 0.0
    for data, target_a, target_b, lam in mixup_loader(train_loader):
        data, target_a, target_b = data.to(device), target_a.to(device), target_b.to(device)

        optimizer.zero_grad()
        outputs = model(data)
        loss = lam * criterion(outputs, target_a) + (1 - lam) * criterion(outputs, target_b)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total += target_a.size(0)
        correct += (lam * predicted.eq(target_a.data).cpu().sum().float() + (1 - lam) * predicted.eq(target_b.data).cpu().sum().float())

    # Print training loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")
    print(f"Training Accuracy: {100 * correct / total}%")

    # Validation
    model.eval()
    correct = 0
    total = 0
    validation_loss = 0
    best_validation_accuracy = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            loss = criterion(outputs, target)
            validation_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += predicted.eq(target.data).cpu().sum().float()

    # Print validation accuracy for each epoch
    val_accuracy = correct / total
    print(f"Validation Loss: {validation_loss / len(val_loader)}")
    print(f"Validation Accuracy: {100 * correct / total}%")

    # Early stopping
    if val_accuracy > best_validation_accuracy:
        best_validation_accuracy = val_accuracy
        no_improvement_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        no_improvement_counter += 1

    if no_improvement_counter >= patience:
        print("Early stopping: No improvement for {} epochs.".format(patience))
        break

# Save the trained model
torch.save(model.state_dict(), 'resnet50_flowers102_mixup.pth')

Epoch 1, Loss: 4.6821287125349045
Training Accuracy: 1.2290819883346558%
Validation Loss: 4.552988916635513
Validation Accuracy: 2.941176414489746%
Epoch 2, Loss: 4.554063558578491
Training Accuracy: 3.552126407623291%
Validation Loss: 4.415706738829613
Validation Accuracy: 10.29411792755127%
Epoch 3, Loss: 4.459620580077171
Training Accuracy: 7.412514686584473%
Validation Loss: 4.291141033172607
Validation Accuracy: 17.54901885986328%
Epoch 4, Loss: 4.337478503584862
Training Accuracy: 16.120710372924805%
Validation Loss: 4.151989296078682
Validation Accuracy: 28.52941131591797%
Epoch 5, Loss: 4.196594052016735
Training Accuracy: 27.983421325683594%
Validation Loss: 3.998853988945484
Validation Accuracy: 40.0%
Epoch 6, Loss: 4.079325683414936
Training Accuracy: 34.222660064697266%
Validation Loss: 3.8673679307103157
Validation Accuracy: 45.882354736328125%
Epoch 7, Loss: 4.030183382332325
Training Accuracy: 37.20534896850586%
Validation Loss: 3.766686365008354
Validation Accuracy: 49.

In [8]:
# Load the saved model (if not already loaded)
model.load_state_dict(torch.load('resnet50_flowers102_mixup.pth'))
model.to(device)
model.eval()  # Set the model to evaluation mode

test_loader = DataLoader(test_dataset, batch_size=batch_size)

test_loss = 0.0
test_correct = 0
test_total = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        test_correct += predicted.eq(target.data).cpu().sum().float()

test_accuracy = 100 * test_correct / total
test_loss /= len(test_loader)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}%")


Test Loss: 1.1083086179328088
Test Accuracy: 83.07041931152344%
